![logo](../picture/license_header_logo.png)

> **Copyright (c) 2020-2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

Authored by: [BK Yeoh](boonkhai.yeoh@certifai.ai)

# Exercise
## Introduction
The data we'll be using is a combined curated dataset of COVID-19 Chest X-ray images obtained by collating 15 publically available datasets. [source](https://data.mendeley.com/datasets/9xkhgts2s6/1)

The dataset contains 1281 COVID-19 X-Rays, 3270 Normal X-Rays, 1656 viral-pneumonia X-Rays, and 3001 bacterial-pneumonia X-Rays. The 4 classes are:
* COVID-19
* Normal
* Pneumonia-Bacterial
* Pneumonia-Viral

Use the model in the previous `image_classifier_exercise` as the base model for this exercise. Feel free to use the helper testing function `utils.Toolbox` to validate your model correctness.

You are required:
* Save your model using:
    * Python Pickle
    * Joblib
    * PyTorch TorchScript
* Save your model format in:
    * ONNX
    * Protocol Buffers
    * JSON format
* Pass the File test
* Pass the Accuracy Test


In [1]:
import numpy as np
from pathlib import Path
import torchvision
from torchvision import transforms,datasets,models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.onnx
from time import time
import warnings
import os
import utils

## Helper Function

In [2]:
# Generate folder to store the model
gen_model_path = 'generated_model/exercise'
dir_list = ['pickle','joblib','torchscript','onnx','protocol_buffers','json'] 
utils.folder_generator(gen_model_path,dir_list)

Directory generated_model/exercise already exists, skipping create
Subfolder pickle already exists, skipping create
Subfolder joblib already exists, skipping create
Subfolder torchscript already exists, skipping create
Subfolder onnx already exists, skipping create
Subfolder protocol_buffers already exists, skipping create
Subfolder json already exists, skipping create


In [3]:
source = 'https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/xray_image_classification-20210604T123548Z-001.zip'
target = '../resources/data/'
filename = 'xray_image_classification.zip'
utils.download(source, target, filename,True)

model_url = "https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/models/xray_classifier_state_dict.pt"
modelname = "xray_classifier_state_dict.pt"
utils.download(model_url, gen_model_path, modelname)

file already exists, skipping download
file already exists, skipping download


In [4]:
# The data is located in the data folder
datadir = Path().resolve().parent/'resources/data/'
dirtytestdir = datadir/'xray_image_classification/test/'

In [5]:
val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [6]:
model = models.squeezenet1_0(pretrained=True)
# Freezing the parameters
for param in model.parameters():
    param.requires_grad = False
    
# Replace a custom classifier to fit our data
torch.manual_seed(123)

num_output = 4
model.classifier = nn.Sequential(nn.Dropout(0.5),
                                 nn.Conv2d(512, num_output, 1, 1),
                                 nn.ReLU(),
                                 nn.AvgPool2d(13, 1))

model.load_state_dict(torch.load(os.path.join(gen_model_path,modelname)))
model

Downloading: "https://download.pytorch.org/models/squeezenet1_0-a815701f.pth" to C:\Users\Sweet/.cache\torch\hub\checkpoints\squeezenet1_0-a815701f.pth


  0%|          | 0.00/4.79M [00:00<?, ?B/s]

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [7]:
'''
Arguments 
model: base model define by user
dirtytestdir: test dataset directory
val_transform: image transforms
'''
toolbox = utils.Toolbox(model,dirtytestdir,val_transform)

## Save & Load the Model using Python Pickle
Save and load your model <br>
`model file path = '../generated_model/exercise/pickle/xray_image_classification.pt'`

In [8]:
import pickle

# save the model to disk
def pickle_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_model_pk

In [9]:
toolbox.test(os.path.join(gen_model_path,dir_list[0]+'/'+"xray_image_classification.pt"),
     pickle_func())

Pass File Test
Testing Accuracy ...
Pass Accuracy Test


## Save & Load the Model using Joblib
Save and load your model <br>
`model file path = '../generated_model/exercise/joblib/xray_image_classification.pt'`<br>

In [10]:
import joblib

# save the model to disk
def joblib_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_model_jb

In [11]:
toolbox.test(os.path.join(gen_model_path,dir_list[1]+'/'+"xray_image_classification.pt"),
     joblib_func())

Pass File Test
Testing Accuracy ...
Pass Accuracy Test


## Save & Load the Model using TorchScript
Use `torch.jit.script` to save you model and `torch.jit.load` to load it.<br>
`model file path = '../generated_model/exercise/torchscrip/xray_image_classification.pt'`<br>

In [12]:
def torch_func(directory = gen_model_path):
    # YOUR CODE HERE
    return loaded_script_model

In [13]:
toolbox.test(os.path.join(gen_model_path,dir_list[2]+'/'+"xray_image_classification.pt"),
     torch_func())

Pass File Test
Testing Accuracy ...
Pass Accuracy Test


## Save Model in ONNX format
Save you model in ONNX format
`model file path = '../generated_model/exercise/onnx/xray_image_classification.onnx'`<br>

In [14]:
x = torch.randn(10, 3, 224, 224, requires_grad=True)
# Export the model
# YOUR CODE HERE

graph(%input : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=1, device=cpu),
      %features.0.weight : Float(96, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %features.0.bias : Float(96, strides=[1], requires_grad=0, device=cpu),
      %features.3.squeeze.weight : Float(16, 96, 1, 1, strides=[96, 1, 1, 1], requires_grad=0, device=cpu),
      %features.3.squeeze.bias : Float(16, strides=[1], requires_grad=0, device=cpu),
      %features.3.expand1x1.weight : Float(64, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %features.3.expand1x1.bias : Float(64, strides=[1], requires_grad=0, device=cpu),
      %features.3.expand3x3.weight : Float(64, 16, 3, 3, strides=[144, 9, 3, 1], requires_grad=0, device=cpu),
      %features.3.expand3x3.bias : Float(64, strides=[1], requires_grad=0, device=cpu),
      %features.4.squeeze.weight : Float(16, 128, 1, 1, strides=[128, 1, 1, 1], requires_grad=0, device=cpu),
      %features.4.s

In [15]:
toolbox.test(os.path.join(gen_model_path,dir_list[3]+'/'+"xray_image_classification.onnx"))

Pass File Test
Testing Accuracy ...
No Accuracy Test for This Question


## Save Model in Protocol Buffers
Save you model in `.pb` format<br>
`model file path = '../generated_model/exercise/protocol_buffers'`<br>

In [16]:
from onnx_tf.backend import prepare
import onnx
warnings.filterwarnings("ignore")
# YOUR CODE HERE
!saved_model_cli show --dir {pb_filepath} --all

INFO:tensorflow:Assets written to: generated_model/exercise\protocol_buffers\assets

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (10, 3, 224, 224)
        name: serving_default_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output'] tensor_info:
        dtype: DT_FLOAT
        shape: (10, 4)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
        Nam

In [17]:
toolbox.test(os.path.join(gen_model_path,dir_list[4]+'/'+"saved_model.pb"))

Pass File Test
Testing Accuracy ...
No Accuracy Test for This Question


## Save model in Json format
Save you model in `.json` format<br>
`model file path = '../generated_model/exercise/json/xray_image_classification.json'`<br>

In [18]:
# YOUR CODE HERE

Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.
Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.
Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.
Unable to use `same` padding. Add ZeroPadding2D layer to fix shapes.


[0, 0, 0, 0, 0, 0, 0, 0]
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 3, 224, 224) 0                                            
__________________________________________________________________________________________________
53 (Conv2D)                     (None, 96, 109, 109) 14208       input[0][0]                      
__________________________________________________________________________________________________
54 (Activation)                 (None, 96, 109, 109) 0           53[0][0]                         
__________________________________________________________________________________________________
55_pad (ZeroPadding2D)          (None, 96, 110, 110) 0           54[0][0]                         
______________________________________________________________

{'class_name': 'Functional',
 'config': {'name': 'functional_1',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': [None, 3, 224, 224],
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input'},
    'name': 'input',
    'inbound_nodes': []},
   {'class_name': 'Conv2D',
    'config': {'name': '53',
     'trainable': True,
     'dtype': 'float32',
     'filters': 96,
     'kernel_size': [7, 7],
     'strides': [2, 2],
     'padding': 'valid',
     'data_format': 'channels_first',
     'dilation_rate': [1, 1],
     'groups': 1,
     'activation': 'linear',
     'use_bias': True,
     'kernel_initializer': {'class_name': 'Zeros', 'config': {}},
     'bias_initializer': {'class_name': 'Zeros', 'config': {}},
     'kernel_regularizer': None,
     'bias_regularizer': None,
     'activity_regularizer': None,
     'kernel_constraint': None,
     'bias_constraint': None},
    'name': '53',
    'inbound_nodes': [[['input', 0, 0, {}]]]},

In [19]:
toolbox.test(os.path.join(gen_model_path,dir_list[5]+'/'+"xray_image_classification.json"))

Pass File Test
Testing Accuracy ...
No Accuracy Test for This Question
